In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
!/opt/conda/bin/pip install scikit-learn

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 3.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 3.6 MB/s eta 0:00:00a 0:00:01


In [4]:
!pwd

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/tmp/pycharm_project_774


In [22]:
df = pd.read_csv("/data/BBox_List_2017.csv")

In [23]:
df.shape

(984, 6)

In [24]:
df.Image.unique().shape

(880,)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Image   984 non-null    object 
 1   Label   984 non-null    object 
 2   x       984 non-null    float64
 3   y       984 non-null    float64
 4   w       984 non-null    float64
 5   h       984 non-null    float64
dtypes: float64(4), object(2)
memory usage: 46.2+ KB


In [26]:
def function(data):
    X = data.drop(["Label"], axis=1).values
    y = data["Label"].values
    
    ss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=0)
    ss.get_n_splits(X, y)
    for i, (train_index, test_index) in enumerate(ss.split(X, y)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
    x_cols = df.drop(["Label"], axis=1).columns.to_list()
    y_cols = df[["Label"]].columns.to_list()
    
    train = pd.DataFrame(data=X_train, columns=x_cols)
    test = pd.DataFrame(data=X_test, columns=x_cols)
    
    train[y_cols[0]] = y_train
    test[y_cols[0]] = y_test
    
    return train,test

In [27]:
train,test = function(df)

In [28]:
train.shape

(688, 6)

In [29]:
test.shape

(296, 6)

In [30]:
train.to_csv("/data/yolo_dataset/train.csv", index=False)
test.to_csv("/data/yolo_dataset/test.csv", index=False)

In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Read the CSV file
df = pd.read_csv('/data/BBox_List_2017.csv')

# Group by Image to handle multiple labels per image
grouped = df.groupby('Image')

# Create lists to store the train and test data
train_data = []
test_data = []

# Iterate through each group (image)
for _, group in grouped:
    # Get unique labels for this image
    labels = group['Label'].unique()
    
    # Create a string of sorted labels to use for stratification
    label_str = '_'.join(sorted(labels))
    
    if len(group) == 1:
        # If there's only one sample, randomly assign it to train or test
        if np.random.random() < 0.7:
            train_data.append(group)
        else:
            test_data.append(group)
    else:
        # Split the group if it has more than one sample
        group_train, group_test = train_test_split(
            group, 
            test_size=0.30, 
            random_state=42, 
            stratify=[label_str] * len(group)
        )
        
        train_data.append(group_train)
        test_data.append(group_test)

# Concatenate all the train and test data
train_df = pd.concat(train_data)
test_df = pd.concat(test_data)

# Verify the split
print(f"Train set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")

# Check the distribution of labels in train and test sets
labels = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltrate', 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']
for label in labels:
    train_count = train_df[train_df['Label'] == label].shape[0]
    test_count = test_df[test_df['Label'] == label].shape[0]
    print(f"{label}:")
    print(f"  Train: {train_count} ({train_count / len(train_df):.2%})")
    print(f"  Test: {test_count} ({test_count / len(test_df):.2%})")

# Save the train and test sets to new CSV files
train_df.to_csv('/data/yolo_dataset/train_set.csv', index=False)
test_df.to_csv('/data/yolo_dataset/test_set.csv', index=False)

Train set size: 661
Test set size: 323
Atelectasis:
  Train: 106 (16.04%)
  Test: 74 (22.91%)
Cardiomegaly:
  Train: 94 (14.22%)
  Test: 52 (16.10%)
Effusion:
  Train: 93 (14.07%)
  Test: 60 (18.58%)
Infiltrate:
  Train: 88 (13.31%)
  Test: 35 (10.84%)
Mass:
  Train: 64 (9.68%)
  Test: 21 (6.50%)
Nodule:
  Train: 55 (8.32%)
  Test: 24 (7.43%)
Pneumonia:
  Train: 92 (13.92%)
  Test: 28 (8.67%)
Pneumothorax:
  Train: 69 (10.44%)
  Test: 29 (8.98%)
